<a href="https://colab.research.google.com/github/omarnabil1516-tech/Inventory_Manegment_Lab02/blob/main/Lab05_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 05 - Web-Based Joint Replenishment Planning Tool

This notebook implements the Decision Support System for the Joint Replenishment Problem (JRP) as described in the assignment PDF.

### How to use:
1. **Run the code cell below** (Click the Play button).
2. The system will automatically create a sample file named `sample_input.json` in the file browser.
3. Use the **User Interface** that appears at the bottom to:
   - Load the sample file (or upload your own).
   - Calculate optimal Cycle Time ($T$) and Multipliers ($m_i$).
   - View the formatted Report and JSON output.

In [1]:
import json
import math
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# ==========================================
# 1. INPUT HANDLING (input_loader.py)
# ==========================================
class InputLoader:
    @staticmethod
    def validate(data):
        """Validates the input dictionary against the required schema."""
        required_keys = ['A', 'r', 'items']
        item_keys = ['id', 'a', 'D', 'v']
        errors = []

        # Check global keys
        for k in required_keys:
            if k not in data:
                errors.append(f"Missing global parameter: {k}")

        if errors: return False, errors

        # Validate numeric types and non-negativity for globals
        if not isinstance(data['A'], (int, float)) or data['A'] < 0:
            errors.append("Major setup cost (A) must be a non-negative number.")
        if not isinstance(data['r'], (int, float)) or data['r'] < 0:
            errors.append("Carrying charge (r) must be a non-negative number.")

        # Validate items
        if not isinstance(data['items'], list) or len(data['items']) == 0:
            errors.append("Items list must be a non-empty list.")
            return False, errors

        for idx, item in enumerate(data['items']):
            for k in item_keys:
                if k not in item:
                    errors.append(f"Item at index {idx} missing key: {k}")

            # Check numeric values for item properties
            for k in ['a', 'D', 'v']:
                if k in item and (not isinstance(item[k], (int, float)) or item[k] < 0):
                    errors.append(f"Item '{item.get('id', idx)}' parameter '{k}' must be non-negative.")

        if errors:
            return False, errors
        return True, "Validation Successful"

# ==========================================
# 2. CORE LOGIC (jrp_solver.py)
# ==========================================
class JRPSolver:
    @staticmethod
    def calculate_total_cost(A, r, items, T, m_list):
        """
        Computes Total Relevant Cost per Unit Time based on formula:
        TC = (A + sum(a_i / m_i)) / T + T/2 * sum(m_i * D_i * v_i * r)
        """
        sum_setup = A
        sum_holding = 0

        for i, item in enumerate(items):
            m_i = m_list[i]
            sum_setup += item['a'] / m_i
            holding_factor = item['D'] * item['v'] * r
            sum_holding += m_i * holding_factor

        ordering_cost = sum_setup / T
        holding_cost = (T * sum_holding) / 2
        total_cost = ordering_cost + holding_cost

        return total_cost, ordering_cost, holding_cost

    @staticmethod
    def solve(data):
        """
        Iterative heuristic to find optimal T and m_i.
        Algorithm:
        1. Initialize all m_i = 1.
        2. Calculate optimal T for fixed m_i.
        3. For fixed T, update m_i for each item to nearest integer that minimizes cost.
        4. Repeat until m_i convergence.
        """
        A = data['A']
        r = data['r']
        items = data['items']
        n = len(items)

        # Initial State: all multipliers = 1
        current_m = [1] * n

        max_iterations = 20
        for iteration in range(max_iterations):
            # Step 1: Calculate optimal T for current multipliers
            # T* = sqrt( 2 * (A + sum(a/m)) / sum(m * D * v * r) )
            numerator_sum = A + sum(item['a'] / current_m[i] for i, item in enumerate(items))
            denominator_sum = sum(current_m[i] * item['D'] * item['v'] * r for i, item in enumerate(items))

            if denominator_sum == 0:
                 T_star = 0 # Avoid division by zero
            else:
                 T_star = math.sqrt((2 * numerator_sum) / denominator_sum)

            # Step 2: Update multipliers based on new T
            # For a given T, optimal m_i (continuous) is approx sqrt( 2*a / (T^2 * D * v * r) )
            new_m = []
            for i, item in enumerate(items):
                H_i = item['D'] * item['v'] * r
                if H_i == 0 or T_star == 0:
                    best_m = 1
                else:
                    # Theoretical continuous m
                    m_theoretical = math.sqrt((2 * item['a']) / (H_i * (T_star**2)))

                    # We must check the integer neighbors (floor and ceil) to minimize local cost function
                    # Local Cost(m) = a/(m*T) + m*T*H/2
                    m_floor = max(1, math.floor(m_theoretical))
                    m_ceil = m_floor + 1

                    cost_floor = (item['a'] / (m_floor * T_star)) + (m_floor * T_star * H_i) / 2
                    cost_ceil = (item['a'] / (m_ceil * T_star)) + (m_ceil * T_star * H_i) / 2

                    if cost_floor < cost_ceil:
                        best_m = int(m_floor)
                    else:
                        best_m = int(m_ceil)

                new_m.append(best_m)

            # Check convergence
            if new_m == current_m:
                break
            current_m = new_m

        # Final calculation
        total_cost, ord_cost, hold_cost = JRPSolver.calculate_total_cost(A, r, items, T_star, current_m)

        # Prepare result structure
        item_results = []
        for i, item in enumerate(items):
            m_i = current_m[i]
            cycle_time = m_i * T_star
            avg_inv = (cycle_time * item['D']) / 2
            item_hold_cost = (cycle_time * item['D'] * item['v'] * r) / 2
            item_setup_cost = item['a'] / cycle_time

            item_results.append({
                "id": item['id'],
                "m": m_i,
                "cycle_time": round(cycle_time, 4),
                "avg_inventory": round(avg_inv, 2),
                "annual_holding_cost": round(item_hold_cost, 2),
                "annual_setup_cost": round(item_setup_cost, 2)
            })

        solution = {
            "T_star": round(T_star, 4),
            "total_annual_cost": round(total_cost, 2),
            "family_setup_cost": round(A / T_star, 2),
            "item_results": item_results
        }
        return solution

# ==========================================
# 3. EXPORT / REPORT (output_exporter.py)
# ==========================================
class OutputExporter:
    @staticmethod
    def generate_text_report(input_data, solution):
        lines = []
        lines.append("=== JOINT REPLENISHMENT PLANNING REPORT ===")
        lines.append(f"Instance Name: {input_data.get('instance_name', 'Unnamed')}")
        lines.append("-" * 40)
        lines.append("INPUT PARAMETERS:")
        lines.append(f"Major Setup Cost (A): ${input_data['A']}")
        lines.append(f"Carrying Charge (r):  {input_data['r']}")
        lines.append(f"Number of Items:      {len(input_data['items'])}")
        lines.append("-" * 40)
        lines.append("OPTIMAL SOLUTION:")
        lines.append(f"Family Basic Cycle (T*): {solution['T_star']} time units")
        lines.append(f"Total Cost per Unit Time: ${solution['total_annual_cost']}")
        lines.append("-" * 40)
        lines.append("{:<10} {:<5} {:<10} {:<12} {:<15}".format("Item ID", "m_i", "Cycle(T_i)", "Avg Inv", "Item Cost"))
        lines.append("-" * 60)

        for item in solution['item_results']:
            total_item_cost = item['annual_holding_cost'] + item['annual_setup_cost']
            lines.append("{:<10} {:<5} {:<10} {:<12} ${:<15}".format(
                str(item['id']), item['m'], str(item['cycle_time']), str(item['avg_inventory']), str(round(total_item_cost, 2))
            ))

        lines.append("-" * 60)
        lines.append("Interpretation:")
        lines.append("Items with higher multipliers (m_i) are replenished less frequently.")
        lines.append("This usually corresponds to items with low demand-value product or high minor setup costs.")
        return "\n".join(lines)

# ==========================================
# 4. WEB/UI LAYER (Simulated for Colab)
# ==========================================

# Create a demo input file on disk so the user has something to start with
demo_data = {
    "instance_name": "Lecture_Example_Replication",
    "A": 15.0,
    "r": 0.2,
    "items": [
        { "id": "Item1", "a": 2.0, "D": 1000.0, "v": 5.0 },
        { "id": "Item2", "a": 2.0, "D": 500.0, "v": 4.0 },
        { "id": "Item3", "a": 3.0, "D": 2000.0, "v": 2.0 },
        { "id": "Item4", "a": 2.0, "D": 100.0, "v": 10.0 }
    ]
}

with open('sample_input.json', 'w') as f:
    json.dump(demo_data, f, indent=4)

# UI Elements
style = {'description_width': 'initial'}
title_lbl = widgets.HTML("<h2>Inventory Planning Cockpit (JRP Solver)</h2>")

file_input_lbl = widgets.Label("1. Load Input Data (JSON):")
# Option to load default
btn_load_sample = widgets.Button(description="Load Sample File", icon='file-text')

# Option to paste JSON directly (easier in Colab than file uploads sometimes)
json_text_area = widgets.Textarea(
    value='',
    placeholder='Paste JSON content here or click Load Sample...',
    description='Input JSON:',
    layout=widgets.Layout(width='100%', height='200px')
)

btn_solve = widgets.Button(description="Solve Instance", button_style='success', icon='play')
output_area = widgets.Output()

def on_load_sample(b):
    with open('sample_input.json', 'r') as f:
        json_text_area.value = f.read()

def on_solve(b):
    with output_area:
        clear_output()
        raw_text = json_text_area.value
        if not raw_text.strip():
            print("Error: Input is empty.")
            return

        try:
            data = json.loads(raw_text)
        except json.JSONDecodeError as e:
            print(f"Error: Invalid JSON Format.\n{e}")
            return

        # Validation
        valid, msgs = InputLoader.validate(data)
        if not valid:
            print("Input Validation Errors:")
            for m in msgs:
                print(f" - {m}")
            return

        # Computation
        try:
            print("Running Optimization...")
            solution = JRPSolver.solve(data)

            # Report
            report = OutputExporter.generate_text_report(data, solution)

            print(report)

            print("\n=== JSON OUTPUT FOR EXPORT ===")
            print(json.dumps(solution, indent=2))

        except Exception as e:
            print(f"An unexpected error occurred during optimization: {e}")

btn_load_sample.on_click(on_load_sample)
btn_solve.on_click(on_solve)

ui = widgets.VBox([
    title_lbl,
    file_input_lbl,
    widgets.HBox([btn_load_sample]),
    json_text_area,
    btn_solve,
    output_area
])

display(ui)